#### imports

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import string as str
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, ReLU, GlobalAvgPool2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.metrics import AUC
import gc

from tensorflow.keras.applications.xception import Xception 

from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications import ResNet50V2

!pip install -U --pre efficientnet
import efficientnet.tfkeras as efn

from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2


In [9]:
img_dim = (224, 224)

n_classes = 10

#### read dataset

In [10]:
df = pd.read_csv('../input/flipkart-dataset-final/final_flipkart.csv')

In [11]:
df.head()

,pid,description,category_x,category_y,path,path1
0,SHOEH4GRSUBJGZXE,Key Features of AW Bellies Sandals Wedges Heel...,footwear,Footwear,Footwear/SHOEH4GRSUBJGZXE.jpg,../input/flipkartcomputervisionuploaded/produc...
1,SRTEH2FVVKRBAXHB,Key Features of Alisha Solid Women's Cycling S...,clothing,Clothing,Clothing/SRTEH2FVVKRBAXHB.jpg,../input/flipkartcomputervisionuploaded/produc...
2,SHOEH3DZBFR88SCK,"Key Features of dilli bazaaar Bellies, Corpora...",footwear,Footwear,Footwear/SHOEH3DZBFR88SCK.jpg,../input/flipkartcomputervisionuploaded/produc...
3,SRTEH2FVUHAAVH9X,Key Features of Alisha Solid Women's Cycling S...,clothing,Clothing,Clothing/SRTEH2FVUHAAVH9X.jpg,../input/flipkartcomputervisionuploaded/produc...
4,SHOEH4KM2W3Z6EH5,Key Features of Ladela Bellies Brand: LADELA C...,footwear,Footwear,Footwear/SHOEH4KM2W3Z6EH5.jpg,../input/flipkartcomputervisionuploaded/produc...


In [12]:
df.isna().sum()

pid            0
description    2
category_x     0
category_y     0
path           0
path1          0
dtype: int64

In [13]:
df.dropna(axis=0, inplace=True)

In [14]:
df.isna().sum()

pid            0
description    0
category_x     0
category_y     0
path           0
path1          0
dtype: int64

In [15]:
df.shape

(12625, 6)

In [16]:
df['category_x'].nunique()

10

In [17]:
df['category_x'].value_counts()

clothing      4014
jewellery     2828
home          1413
footwear      1001
mobiles        870
automotive     809
beauty         568
kitchen        508
computers      460
watches        154
Name: category_x, dtype: int64

#### encoding

In [18]:
encoder = LabelEncoder()

df['label_encoded'] = encoder.fit_transform(df['category_x'])

#### train-test split

In [19]:
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['category_x'], random_state=22)

In [20]:
df_train.shape, df_test.shape

((10100, 7), (2525, 7))

In [21]:
train_images = np.array([cv2.resize(cv2.imread(path), img_dim) for path in df_train['path1']])

test_images = np.array([cv2.resize(cv2.imread(path), img_dim) for path in df_test['path1']])

In [22]:
train_images.shape, test_images.shape

((10100, 224, 224, 3), (2525, 224, 224, 3))

In [23]:
train_labels = df_train['label_encoded'].values

test_labels = df_test['label_encoded'].values

In [24]:
train_labels = train_labels.reshape(-1, 1)

test_labels = test_labels.reshape(-1, 1)

In [25]:
train_labels.shape, test_labels.shape

((10100, 1), (2525, 1))

In [26]:
train_labels_encoded = to_categorical(train_labels, num_classes=n_classes)

test_labels_encoded = to_categorical(test_labels, num_classes=n_classes)

In [27]:
train_labels_encoded.shape, test_labels_encoded.shape

((10100, 10), (2525, 10))

##### mobilenet

In [28]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_dim[0], img_dim[1], 3))

base_model.trainable = False

2022-03-25 12:43:14.248920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


17235968/17225924 [==============================] - 0s 0us/step


2022-03-25 12:43:14.341043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 12:43:14.341716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 12:43:14.343015: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-25 12:43:14.343813: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [29]:
stopping = EarlyStopping(monitor='val_loss', 
                         patience=3, 
                         verbose=0, 
                         restore_best_weights=True)

checkpoint = ModelCheckpoint('best_weights.hdf5', 
                              monitor='val_loss', 
                              verbose=1, 
                              save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2,
                              verbose=1,
                              patience=2, 
                              min_lr=0.001)

In [30]:
model = Sequential()

model.add(base_model)

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))

# model.add(Dense(units=128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(rate=0.3))

model.add(Dense(units=n_classes, activation='softmax'))  

#### evaluation metrics

In [31]:
import keras.backend as K

def f1_metric_custom(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1_val

In [32]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=[f1_metric_custom, F1Score(num_classes=n_classes, average='macro')])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               12845312  
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 16,077,770
Trainable params: 12,848,394
Non-trainable params: 3,229,376
____________________________________

In [34]:
model_hist = model.fit(train_images, train_labels_encoded, validation_split=0.2, epochs=20, batch_size=64, workers=8, callbacks=[checkpoint, stopping, reduce_lr])  

2022-03-25 12:43:17.829629: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1216266240 exceeds 10% of free system memory.
2022-03-25 12:43:19.085464: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1216266240 exceeds 10% of free system memory.
2022-03-25 12:43:19.998965: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-03-25 12:43:23.295529: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


127/127 [==============================] - 16s 62ms/step - loss: 0.7742 - f1_metric_custom: 0.7874 - f1_score: 0.6856 - val_loss: 0.4458 - val_f1_metric_custom: 0.8802 - val_f1_score: 0.8138

Epoch 00001: val_loss improved from inf to 0.44582, saving model to best_weights.hdf5
Epoch 2/20
127/127 [==============================] - 7s 53ms/step - loss: 0.2143 - f1_metric_custom: 0.9396 - f1_score: 0.9220 - val_loss: 0.4469 - val_f1_metric_custom: 0.8818 - val_f1_score: 0.8167

Epoch 00002: val_loss did not improve from 0.44582
Epoch 3/20
127/127 [==============================] - 7s 53ms/step - loss: 0.0860 - f1_metric_custom: 0.9809 - f1_score: 0.9770 - val_loss: 0.3928 - val_f1_metric_custom: 0.8975 - val_f1_score: 0.8405

Epoch 00003: val_loss improved from 0.44582 to 0.39276, saving model to best_weights.hdf5
Epoch 4/20
127/127 [==============================] - 7s 53ms/step - loss: 0.0388 - f1_metric_custom: 0.9943 - f1_score: 0.9949 - val_loss: 0.3643 - val_f1_metric_custom: 0.9061

#### result

In [35]:
print(f'F1 score: {model.evaluate(test_images, test_labels_encoded)[1]*100}')

79/79 [==============================] - 2s 24ms/step - loss: 0.4303 - f1_metric_custom: 0.8883 - f1_score: 0.8335
F1 score: 88.83417844772339
